<a href="https://colab.research.google.com/github/ludimilalustosa-afk/blank-app/blob/main/Tarefa2PortifolioEficiente_AEDI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Autor     : Prof João Gabriel de Moraes Souza
Adaptação : Ludimila Lustosa Guimarães LOpes

Instalação do yahooquery para utilização dos dados do Yahoo Finance
"""
!pip install yahooquery plotly pillow scipy

"""
Importação das bibliotecas
"""
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import norm
from yahooquery import Ticker
from datetime import date
from pandas.tseries.offsets import BDay

"""
Função para buscar dados históricos do Yahoo Finance
"""
def load_data(tickers, start_date, end_date):
    try:
        tq = Ticker(tickers, asynchronous=False)
        hist = tq.history(
            start=start_date.strftime('%Y-%m-%d'),
            end=end_date.strftime('%Y-%m-%d')
        )

        if hist.empty:
            print("⚠️ Nenhum dado retornado. Verifique tickers e datas.")
            return pd.DataFrame()

        df = hist.reset_index().pivot(index='date', columns='symbol', values='close')
        df.reset_index(inplace=True)
        df.rename(columns={col: col.upper() for col in df.columns}, inplace=True)
        df.rename(columns={'DATE': 'Date'}, inplace=True)
        return df

    except Exception as e:
        print(f"Erro ao buscar dados: {e}")
        return pd.DataFrame()


def get_ret_vol_sr(weights, log_ret):
    weights = np.array(weights)
    ret = np.sum(log_ret.mean() * weights * 252)
    vol = np.sqrt(np.dot(weights.T, np.dot(log_ret.cov() * 252, weights)))
    sr = ret / vol
    return np.array([ret, vol, sr])


def alocacao_ativos(dataset, dinheiro_total, seed=0, melhores_pesos=[]):
    dataset = dataset.copy()

    if seed != 0:
        np.random.seed(seed)

    if len(melhores_pesos) > 0:
        pesos = melhores_pesos
    else:
        pesos = np.random.random(len(dataset.columns) - 1)
        pesos = pesos / np.sum(pesos)

    colunas = dataset.columns[1:]

    for i, acao in enumerate(colunas):
        dataset[acao] = (dataset[acao] / dataset[acao][0]) * pesos[i] * dinheiro_total

    dataset['soma valor'] = dataset[colunas].sum(axis=1)
    dataset['taxa retorno'] = dataset['soma valor'].pct_change() * 100

    patrimonio_df = pd.DataFrame({'Date': dataset['Date'], 'Patrimônio': dataset['soma valor']})
    return dataset, patrimonio_df, dataset['soma valor'].iloc[-1]


def calcular_var(returns, confidence_level):
    returns = np.array(returns)
    z_score = norm.ppf(confidence_level)
    stdev = np.std(returns)
    var = -(returns.mean() + z_score * stdev)
    return var



"""
### para pegar o último dia útil do ano atual
hoje = pd.Timestamp.today().normalize()
ultimo_dia_util = (hoje - BDay(1)).date()
"""




"""
tickers: são as ações escolhidas para análise
"""
tickers = ['EQTL3.SA', 'LREN3.SA', 'ITUB4.SA', 'PETR4.SA', 'VALE3.SA']

"""
Intervalo de datas para análise
end_date   = ultimo_dia_util      ### se for usar o último dia útil do ano atual
"""
start_date = pd.to_datetime('2015-08-01').date()
end_date   = pd.to_datetime('2025-08-31').date()


print("\n⏳ Carregando dados do Yahoo Finance do período de" ,start_date , "à", end_date,"\n")

print("\nDADOS DAS AÇÕES NO PERÍODO DE",start_date , "À", end_date)
data = load_data(tuple(tickers), start_date, end_date)
# Prévia dos dados
display(data.head())


"""
Gráfico: Histórico dos Preços
"""
fig = px.line(data, x='Date', y=tickers, title='Histórico dos Preços')
fig.show()

"""
Gráfico: Histórico de Retorno das Ações
"""
log_retornos = np.log(data.set_index('Date') / data.set_index('Date').shift(1))
fig_retornos = px.line(log_retornos, title="Histórico de Retorno das Ações")
fig_retornos.show()

"""
Correlação das taxas de retorno
"""
print("\nCORRELAÇÕES DAS TAXAS DE RETORNO")
correlacao = log_retornos.corr()
# Arredondando os valores para 2 casas decimais
correlacao = np.round(correlacao, 2)
###correlacao
# Prévia dos dados
display(correlacao.head())
print("\n")


"""
Gráfico de calor de correlação
"""
### Definindo o esquema de cores com vermelho em valores altos (próximo a 1)
custom_colorscale = [
    [0.0, 'green'],   # Cor para o limite inferior (-1)
    [0.5, 'blue'],    # Cor para o valor neutro (0)
    [1.0, 'red']      # Cor para o limite superior (1)
]

### Criando o heatmap com o esquema de cores customizado
fig = px.imshow(correlacao,
                text_auto=True,
                aspect="auto",
                title="Mapa de Calor de Correlações",
                color_continuous_scale=custom_colorscale,  # Esquema de cores customizado
                labels=dict(color="Correlações"),
                zmin=-1, zmax=1)  # Definindo o limite de correlações
fig.show()



"""
Simulação de Portfólio
simulações:                     50.000
valor incial de investimento:   35.000,00
"""
num_ports = 50000
valor_inicial = 35000

print("🔄 Rodando simulação de portfólio...")

data.dropna(inplace=True)
log_ret = np.log(data.set_index('Date') / data.set_index('Date').shift(1))
log_ret.reset_index(drop=True, inplace=True)

np.random.seed(42)
n_assets = len(data.columns) - 1 if 'Date' in data.columns else len(data.columns)
all_weights = np.zeros((num_ports, n_assets))
ret_arr = np.zeros(num_ports)
vol_arr = np.zeros(num_ports)
sharpe_arr = np.zeros(num_ports)

for x in range(num_ports):
    weights = np.random.random(n_assets)
    weights = weights / np.sum(weights)
    all_weights[x, :] = weights
    ret_arr[x], vol_arr[x], sharpe_arr[x] = get_ret_vol_sr(weights, log_ret)

max_sr_loc = sharpe_arr.argmax()
max_sr_ret = ret_arr[max_sr_loc]
max_sr_vol = vol_arr[max_sr_loc]

"""
Gráfico de Simulação de Portifólios
"""
fig_port = px.scatter(
    x=vol_arr,
    y=ret_arr,
    color=sharpe_arr,
    color_continuous_scale='viridis',
    labels={'x': 'Volatilidade','y': 'Retorno'},
    title='Simulação de Portfólio'
)
"""
Cálculo do Melhor Sharpe Ratio
"""
fig_port.add_scatter(
    x=[max_sr_vol],
    y=[max_sr_ret],
    marker=dict(color='red', size=20),
    name='Melhor Sharpe Ratio'
)
fig_port.show()

"""
Tabela de Pesos do Melhor Portifólio
"""
ativos = data.columns[1:]
pesos = all_weights[max_sr_loc] * 100
df_pesos = pd.DataFrame({'Ativo': ativos, 'Peso (%)': pesos})
print("\nPESOS DO MELHOR PORTIFÓLIO")
display(df_pesos)

"""
Cálculo do Value at Risk (VaR)
"""
confidence_level = 0.95
returns = np.random.normal(max_sr_ret * 100, max_sr_vol * 100, 10000)
var_value = calcular_var(returns, confidence_level)

print(f"\n🔒 VaR ({confidence_level*100:.0f}% confiança): {var_value:.2f}%")

fig_var = go.Figure()
fig_var.add_trace(go.Histogram(
    x=returns,
    nbinsx=50,
    histnorm='probability',
    name='Distribuição de Retornos',
    marker=dict(color='skyblue'),
    opacity=0.75
))
fig_var.add_shape(
    type="line",
    x0=var_value,
    y0=0,
    x1=var_value,
    y1=0.1,
    line=dict(color="red", width=2, dash="dash")
)
fig_var.update_layout(
    title=f'Distribuição dos Retornos com VaR até {confidence_level * 100:.0f}%',
    xaxis_title="Retorno Percentual",
    yaxis_title="Densidade",
    bargap=0.2,
)
fig_var.show()

!jupyter nbconvert --to html Tarefa2PortifolioEficiente_AEDI.ipynb



⏳ Carregando dados do Yahoo Finance do período de 2015-08-01 à 2025-08-31 


DADOS DAS AÇÕES NO PERÍODO DE 2015-08-01 À 2025-08-31


symbol,Date,EQTL3.SA,ITUB4.SA,LREN3.SA,PETR4.SA,VALE3.SA
0,2015-08-03,6.997721,16.484848,14.117330,10.03,17.59
1,2015-08-04,6.999718,16.088154,14.277246,10.16,18.10
2,2015-08-05,7.055652,16.060606,14.243366,10.01,19.01
3,2015-08-06,6.865876,16.000000,14.018399,10.34,19.58
4,2015-08-07,6.746018,15.531680,13.727027,9.69,18.42



CORRELAÇÕES DAS TAXAS DE RETORNO


symbol,EQTL3.SA,ITUB4.SA,LREN3.SA,PETR4.SA,VALE3.SA
symbol,,,,,
EQTL3.SA,1.00,0.47,0.50,0.33,0.20
ITUB4.SA,0.47,1.00,0.52,0.52,0.31
LREN3.SA,0.50,0.52,1.00,0.36,0.22
PETR4.SA,0.33,0.52,0.36,1.00,0.43
VALE3.SA,0.20,0.31,0.22,0.43,1.00


🔄 Rodando simulação de portfólio...
